In [1]:
import torch

# Numpy 实现梯度下降

In [5]:
## 设置超参数
x = 0 # 初始值
lr = 0.1 # 学习率
epochs = 10 # 迭代次数

## 定义函数y
y = lambda x: x**2+2*x+1

## 梯度下降算法求解
for epoch in range(epochs):
    dx = 2*x+2 # d(y)/d(x)
    x -= lr * dx #梯度下降法

In [6]:
print(x)
print(y)

-0.8926258176
<function <lambda> at 0x000000000650D840>


# Pytorch 实现梯度下降

In [7]:
from torch.autograd import Variable

# 定义一个pytorch类型 且可自动求导的的初始值
x = torch.Tensor([1])# 定义一个tensor，相当于np.array
x = Variable(x,requires_grad=True) # x转变为一个variable，建立计算图的起点;开启requires_grad表示自动计算梯度

print('grad',x.grad,'data',x.data) # grad表示x的梯度属性，表明当前累计的梯度；data表示tensor值

grad None data tensor([1.])


In [9]:
lr = 0.1
epochs = 10

for epoch in range(epochs):
    # 设置计算图:建立一个函数y，以x为变量
    y = x**2+2*x+1
    # Variable 能自动求导==》requires_grad
    y.backward() # 对y做反向传导==》自动计算梯度，由于当前变量为1个，所以不需要指定
    print('grad of epoch'+str(epoch)+':',x.grad.data)
    
    x.data -= lr * x.grad.data
    # 在 pytorch 中梯度会累积，则每次需要清0
    x.grad.data.zero_() #xx_表示对变量做inplace操作；此处将当前梯度清0
print(x.data)

grad of epoch0: tensor([4.])
grad of epoch1: tensor([3.2000])
grad of epoch2: tensor([2.5600])
grad of epoch3: tensor([2.0480])
grad of epoch4: tensor([1.6384])
grad of epoch5: tensor([1.3107])
grad of epoch6: tensor([1.0486])
grad of epoch7: tensor([0.8389])
grad of epoch8: tensor([0.6711])
grad of epoch9: tensor([0.5369])
tensor([-0.7853])


## Numpy 做回归

In [30]:
import numpy as np

# 建立简单的数据集
f = lambda x:2*(x**2)+3*x+4
n=3
x_data = np.arange(n).reshape(1,n)
y_data = np.array([f(i) for i in x_data])

epochs=400
lr = 0.1
w = np.ones((3,1),dtype='float64')
cost=[]

for epoch in range(epochs):
    # 计算梯度
    x = np.concatenate((x_data**2,x_data))
    x = np.concatenate((x,np.ones((1,n)))) ## 3*10
    yhat = np.dot(w.T,x) ## 设计模型，用x预测y
    loss = np.average((y_data-yhat)**2) # loss函数使用MSE
    cost.append(loss)
    dw = (-2/y_data.shape[1])*np.dot(x,(y_data-yhat).T)
    w -= dw * lr
print('w:',w)

w: [[2.01161267]
 [2.97488811]
 [4.00599999]]


## pytorch 实现回归

In [33]:
import torch
from torch.autograd import Variable

In [57]:
torch.manual_seed(2) # 设置随机种子

# 设置初始变量
x_data = Variable(torch.Tensor([[1],[2],[3]]))
y_data = Variable(torch.Tensor([[2],[4],[6]]))

epochs=20
lr = 0.1
w = Variable(torch.FloatTensor([0]),requires_grad = True) ## 不要忘记requires_grad
cost=[]

for epoch in range(epochs):
    # 计算梯度
    yhat = x_data * w
    loss = torch.mean((yhat - y_data)**2)
    cost.append(loss.data.numpy())
    loss.backward() #计算偏导
    
    # 更新参数
    w.data -= lr*w.grad.data
    w.grad.data.zero_()
print(w.data)

tensor([2.])


## pytorch实现简单的神经网络

In [59]:
# 用pytorh求导来进行
import torch
from torch.autograd import Variable


torch.manual_seed(2)
x_data = Variable(torch.Tensor([[1],[2],[3]]))
y_data = Variable(torch.Tensor([[2],[4],[6]]))

In [66]:
## 定义模型
# class构建一个类，通过class Model写一个神经网络类
class Model(torch.nn.Module):
    # 初始化
    def __init__(self):
        super(Model,self).__init__()
        # super 用来返回Model的父类，在pytorch下定义的类都是继承一个大的父类torch.nn.Module的父类。
        # torch.nn.Module中包含了各种工具，一般我们都是写的都是子类，通过父类我们可以很容易书写子类。
        self.linear = torch.nn.Linear(1,1,bias=False)
        # 建立一个linear类，bias表示偏置项,建立一个AX+b
        
        # forward 是torch.nn.Module定义好的模板，表示前向传播
    def forward(self,x):
        y_pred = self.linear(x)
        return y_pred
model = Model() # 实例化类

In [69]:
## 定义损失函数
criterion = torch.nn.MSELoss(size_average=True) # 平方误差损失

C:\Users\LENOVO\anaconda\lib\site-packages\torch\nn\_reduction.py:49: UserWarning: size_average and reduce args will be deprecated, please use reduction='mean' instead.
  warnings.warn(warning.format(ret))


In [70]:
## 定义优化函数
optimizer = torch.optim.SGD(model.parameters(),lr=0.01) #利用梯度下降算法优化model.parameters

In [72]:
## 优化过程
epochs = 30
cost=[]
for epoch in range(epochs):
    # 建立计算图
    y_hat = model(x_data) #预测值
    loss = criterion(y_hat,y_data)
    cost.append(loss.data)
    optimizer.zero_grad()# 对模型参数做一个优化，并且将梯度清0
    loss.backward()# 计算梯度
    
    ## 参数更新
    optimizer.step()
list(model.parameters())

[Parameter containing:
 tensor([[1.9871]], requires_grad=True)]